In [25]:
import numpy as np
import pandas as pd
from datetime import datetime

df1 = pd.read_csv("SKM Forbruksprognose 2014-2018.csv")
df2 = pd.read_csv("SKM Forbruksprognose 2018-dd.csv")

df1["#Hour"] = pd.to_datetime(df1["#Hour"])
df2["#Hour"] = pd.to_datetime(df2["#Hour"])

df1 = df1.set_index("#Hour")
df2 = df2.set_index("#Hour")

df1 = df1.drop(["CNPNO5_F"],axis=1)
df2 = df2.drop(["CNPNO5_F"],axis=1)

df = pd.concat([df1,df2])

df = df[~(df.index < '2015-01-01')]
df = df[~(df.index > '2022-06-03')]

df["CNPNO1_F"] = df["CNPNO1_F"].apply(lambda x: x*1000)

df

,CNPNO1_F
#Hour,
2015-01-01 00:00:00,4780.0
2015-01-01 01:00:00,4670.0
2015-01-01 02:00:00,4580.0
2015-01-01 03:00:00,4510.0
2015-01-01 04:00:00,4480.0
...,...
2022-05-07 19:00:00,2610.0
2022-05-07 20:00:00,2570.0
2022-05-07 21:00:00,2520.0


In [26]:
import os
from msal import PublicClientApplication

from cognite.client import CogniteClient
from cognite.client.data_classes import TimeSeries, Asset

# Log-in detaljer
TENANT_ID = os.getenv("AZURE_TENANT_ID")
CLIENT_ID = os.getenv("AZURE_CLIENT_ID")
CDF_CLUSTER = "az-power-no-northeurope"
COGNITE_PROJECT = "heco-dev"

# Code to log-in WIHTOUT client_secret
SCOPES = [f"https://{CDF_CLUSTER}.cognitedata.com/.default"]

AUTHORITY_HOST_URI = "https://login.microsoftonline.com"
AUTHORITY_URI = AUTHORITY_HOST_URI + "/" + TENANT_ID
PORT = 53000


def authenticate_azure():

    app = PublicClientApplication(client_id=CLIENT_ID, authority=AUTHORITY_URI)

    # interactive login - make sure you have http://localhost:port in Redirect URI in App Registration as type "Mobile and desktop applications"
    creds = app.acquire_token_interactive(scopes=SCOPES, port=PORT)
    return creds


creds = authenticate_azure()

client = CogniteClient(
    token_url=creds["id_token_claims"]["iss"],
    token=creds["access_token"],
    token_client_id=creds["id_token_claims"]["aud"],
    project=COGNITE_PROJECT,
    base_url=f"https://{CDF_CLUSTER}.cognitedata.com",
    client_name="cognite-python-dev",
)

In [27]:
#Definer ønsket kruver
curves = ["NO1_consumption_per_15min"]

#Definer start dato
start_dato = datetime(2015,1,1)
slutt_dato = datetime(2022,5,8)

df_watt = pd.DataFrame()
for curve in curves:
    print(curve)
    hm = client.datapoints.retrieve_dataframe(
        start=start_dato,
        end=slutt_dato,
        aggregates=["average"],
        granularity="1h",
        id=client.time_series.retrieve(external_id=curve).id,)
    df_watt = pd.merge(df_watt, hm, left_index=True, right_index=True, how="outer")

df_watt

NO1_consumption_per_15min


,5172002421720946|average
2015-01-01 00:00:00,4678.090390
2015-01-01 01:00:00,4574.095914
2015-01-01 02:00:00,4498.004329
2015-01-01 03:00:00,4480.165823
2015-01-01 04:00:00,4540.027045
...,...
2022-05-07 19:00:00,2913.250000
2022-05-07 20:00:00,2921.750000
2022-05-07 21:00:00,2856.625000
2022-05-07 22:00:00,2744.875000


In [28]:
df_watt = pd.merge(df,df_watt,left_index=True,right_index=True)

df_watt

,CNPNO1_F,5172002421720946|average
2015-01-01 00:00:00,4780.0,4678.090390
2015-01-01 01:00:00,4670.0,4574.095914
2015-01-01 02:00:00,4580.0,4498.004329
2015-01-01 03:00:00,4510.0,4480.165823
2015-01-01 04:00:00,4480.0,4540.027045
...,...,...
2022-05-07 19:00:00,2610.0,2913.250000
2022-05-07 20:00:00,2570.0,2921.750000
2022-05-07 21:00:00,2520.0,2856.625000
2022-05-07 22:00:00,2470.0,2744.875000


In [29]:
df_watt.columns = ["cons_forecast","cons_actual"]

df_watt

,cons_forecast,cons_actual
2015-01-01 00:00:00,4780.0,4678.090390
2015-01-01 01:00:00,4670.0,4574.095914
2015-01-01 02:00:00,4580.0,4498.004329
2015-01-01 03:00:00,4510.0,4480.165823
2015-01-01 04:00:00,4480.0,4540.027045
...,...,...
2022-05-07 19:00:00,2610.0,2913.250000
2022-05-07 20:00:00,2570.0,2921.750000
2022-05-07 21:00:00,2520.0,2856.625000
2022-05-07 22:00:00,2470.0,2744.875000


In [30]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score

df_watt = df_watt.dropna()

cons_actual = df_watt["cons_actual"].values
cons_forecast = df_watt["cons_forecast"].values

# cons_actual = cons_actual[~np.isnan(cons_actual)]
# cons_forecast = cons_forecast[~np.isnan(cons_forecast)]

print(f"MSE Volue: {mean_squared_error(cons_forecast, cons_actual)}")
print(f"MAE Volue: {mean_absolute_error(cons_forecast, cons_actual)}")
print(f"MSLE Volue: {mean_squared_log_error(cons_forecast, cons_actual)}")
print(f"R2 Volue: {r2_score(cons_forecast, cons_actual)}")

MSE Volue: 1403891.846651242
MAE Volue: 716.2967293661836
MSLE Volue: 0.084158821779012
R2 Volue: 0.1867852173243153
